<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/Web_scraping_Accidents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install beautifulsoup4
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 9.6MB/s 


In [0]:
import sys
from datetime import *
from time import *

def get_yobi(inDate):
  
  yobi = ["月","火","水","木","金","土","日"]

  a = datetime.strptime(inDate,'%Y/%m/%d')

  #return yobi[a.weekday()]
  return a.weekday()

In [0]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
#import MeCab
from google.colab import files

#https://accident.laboneko.jp/accidents?limit=10&offset=0
#https://accident.laboneko.jp/accidents?limit=11680&offset=0

# 親ページURL
url_pr = 'https://accident.laboneko.jp/accidents?limit=11680&offset=0'

# データフレームを定義
columns = ["date","time","yobi","line","from_st","to_st","st"]
df = pd.DataFrame(columns=columns)

html_doc = urlopen(url_pr)

# BeautifulSoupでスクレイピング
soup = BeautifulSoup(html_doc, 'html.parser')
# ラッピング単位のクラスを取得
table  = soup.find_all("table", {"class": "table table-striped"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_date = dataRow[0][0:10]
    v_time = dataRow[0][11:16]
    v_yobi = get_yobi(v_date)

    v_line = dataRow[1]

    v_st_info = dataRow[2]
    v_st_info_sp = v_st_info.split("〜")

    if len(v_st_info_sp) >= 2:
      v_from_st = v_st_info_sp[0]
      v_to_st = v_st_info_sp[1]
      v_st = v_st_info_sp[0]
    else:
      v_from_st = v_st_info
      v_to_st = v_st_info
      v_st = v_st_info

    # 各データをデータフレームに格納
    se = pd.Series([v_date,v_time,v_yobi,v_line,v_from_st,v_to_st,v_st], columns)
    df = df.append(se, columns)

# 収集したデータをCSV形式で保存
filename = "Accident_list.csv"
df.to_csv(filename, encoding = 'utf-8',index=False)
#files.download(filename)


##ファイルがあればここから実行

In [0]:
import pandas as pd

df_ac = pd.read_csv("Accident_list.csv")

#鉄道会社の人身事故発生順にソート
df_line = df_ac["line"].value_counts().reset_index()

#SeriseをDataFrameに変換
col_name = ["line","count"]
df_line = pd.DataFrame(df_line)

df_line.columns = col_name
df_line.head()

,line,count
0,東武東上線,291
1,山陽本線_(JR西日本),263
2,京浜東北線,245
3,中央快速線,244
4,東海道線_(JR東海),244


In [0]:
#件数は削除
df_line = df_line.drop("count",axis=1)
df_line

#CSVに出力
df_line.to_csv("line_List_Candidate.csv", encoding = 'utf-8',index=False,header=False)

In [0]:
import itertools

kanto_line = pd.read_table('kanto_line.txt', header=None)

#dataframeをlist化
kanto_line_list = kanto_line.values.tolist()
#2次元listから1次元listへ
kanto_line_list = list(itertools.chain.from_iterable(kanto_line_list))

print(kanto_line_list)

['東武東上線', '京浜東北線', '中央快速線', '常磐線', '小田急小田原線', '西武新宿線', '宇都宮線', '京王線', '京急本線', '西武池袋線', '東武伊勢崎線', '東海道線_(JR東日本)', '高崎線', '中央・総武緩行線', '東急田園都市線', '山手線', '東北本線', '近鉄大阪線', '京成本線', '横浜線', '横須賀線', '総武快速線', '南武線', '埼京線', '相鉄本線', '東武野田線', '東急東横線', '外房線', '新京成電鉄新京成線', '中央本線_(JR東日本)', '小田急江ノ島線', '武蔵野線', '総武本線', '都電荒川線', '東京メトロ東西線', '青梅線', '関東鉄道常総線', '成田線', '根岸線', '京王井の頭線', '都営地下鉄新宿線', '京葉線', '秩父鉄道秩父本線', '東急大井町線', '東京メトロ千代田線', '八高線', '東急池上線', '京王相模原線', '川越線', '東京メトロ日比谷線', '西武拝島線', '京成押上線', '相模線', '都営地下鉄浅草線', '江ノ島電鉄線', '東武スカイツリーライン', '東京メトロ半蔵門線', '東武アーバンパークライン', '都営地下鉄大江戸線', '東葉高速鉄道東葉高速線', '東京メトロ銀座線', '小田急多摩線', '横浜高速鉄道みなとみらい線', 'JR東西線', '首都圏新都市鉄道つくばエクスプレス', '京急空港線', '相鉄いずみ野線', '東京メトロ有楽町線', '東京臨海高速鉄道りんかい線', '鶴見線', '東京モノレール羽田空港線', '東武亀戸線', '京葉臨海鉄道臨海本線', '京王新線', '西武秩父線', '埼玉高速鉄道線', '横浜市営地下鉄ブルーライン']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#関東(首都圏)に影響を与えそうな路線のデータのみ抽出
new_data = df_ac[df_ac['line'].isin(kanto_line_list)]
new_data = new_data.reset_index()

#不要な項目を削除
new_data = new_data.drop("index",axis=1)

new_data.head(5)

,date,time,yobi,line,from_st,to_st,st
0,2019/08/10,20:08,5,八高線,拝島駅,東福生駅,拝島駅
1,2019/08/09,22:39,4,東急東横線,学芸大学駅,学芸大学駅,学芸大学駅
2,2019/08/07,18:00,2,宇都宮線,東大宮駅,蓮田駅,東大宮駅
3,2019/08/06,11:54,1,総武本線,八街駅,日向駅,八街駅
4,2019/08/05,12:18,0,東急大井町線,自由が丘駅,九品仏駅,自由が丘駅


In [0]:
!pip install pandas-profiling

In [0]:
import pandas_profiling as pdp
from IPython.display import HTML

profile = pdp.ProfileReport(new_data)
profile.to_file(outputfile="myOutputFile_line.html")

#html化して表示
HTML(filename='myOutputFile_line.html')

Number of variables,7
Number of observations,5154
Total Missing (%),0.0%
Total size in memory,281.9 KiB
Average record size in memory,56.0 B
Numeric,1
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0


# **profilingで分かったこと**
・曜日は関係無さそう

・分析する上で情報が不足している

・正規化、細分化できる情報は無いか

In [0]:
# データフレームを定義
#通勤時間帯/帰宅時間帯/日中/それ以外で分けてみよう

#コピーを付けないとコピー元も変更されてしまう
new_df = new_data.copy()

#カラム「time」をコピーして追加
#(後でカテゴリ分けしたい為)
new_df = new_df.assign(time_category=new_df['time'])
new_df = new_df.assign(season=new_df['date'])

#-------------------------------------------------------------------------------------------
#時間帯
#-------------------------------------------------------------------------------------------
#数値型で比較したい場合は「.str[0:2].astype(int)」を使用する
new_df.loc[(new_df['time_category'].str[0:2].isin(["04","05"])) ,"time_category"] = "morning_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["06","07","08","09"])) ,"time_category"] = "comp_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["17","18","19","20","21"])) ,"time_category"] = "home_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["22","23","00","01"])) ,"time_category"] = "night_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["02","03","10","11","12","13","14","15","16"])) ,"time_category"] = "etc_time"

dmy_df = pd.get_dummies(new_df['time_category'])
add_df = pd.merge(new_df, dmy_df, left_index=True, right_index=True)
add_df = add_df.drop("time_category",axis=1)

#-------------------------------------------------------------------------------------------
#気候
#-------------------------------------------------------------------------------------------
add_df.loc[(add_df['season'].str[5:7].isin(["04","05","10","11"])) ,"season"] = "warm"
add_df.loc[(add_df['season'].str[5:7].isin(["06","07","08","09"])) ,"season"] = "hot"
add_df.loc[(add_df['season'].str[5:7].isin(["12","01","02","03"])) ,"season"] = "cold"

dmy_df = pd.get_dummies(add_df['season'])
add_df = pd.merge(add_df, dmy_df, left_index=True, right_index=True)
add_df = add_df.drop("season",axis=1)

#-------------------------------------------------------------------------------------------
#曜日
#-------------------------------------------------------------------------------------------
dmy_df = pd.get_dummies(add_df['yobi'],prefix='yobi')
add_df = pd.merge(add_df, dmy_df, left_index=True, right_index=True)
add_df = add_df.drop("yobi",axis=1)

add_df.head(5)

,date,time,line,from_st,to_st,st,comp_time,etc_time,home_time,morning_time,night_time,cold,hot,warm,yobi_0,yobi_1,yobi_2,yobi_3,yobi_4,yobi_5,yobi_6
0,2019/08/10,20:08,八高線,拝島駅,東福生駅,拝島駅,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0
1,2019/08/09,22:39,東急東横線,学芸大学駅,学芸大学駅,学芸大学駅,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0
2,2019/08/07,18:00,宇都宮線,東大宮駅,蓮田駅,東大宮駅,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,2019/08/06,11:54,総武本線,八街駅,日向駅,八街駅,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0
4,2019/08/05,12:18,東急大井町線,自由が丘駅,九品仏駅,自由が丘駅,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0


In [0]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

# 親ページURL
url_pr = 'https://www.jreast.co.jp/passenger/index.html'

# データフレームを定義
columns = ["st","ninzu"]
df_ppl = pd.DataFrame(columns=columns)

html_doc = urlopen(url_pr)

# BeautifulSoupでスクレイピング
soup = BeautifulSoup(html_doc, 'html.parser')
# ラッピング単位のクラスを取得1
table  = soup.find_all("table", {"class": "passengerTable fl"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_stname = dataRow[1]+"駅"
    v_ppl = dataRow[4].replace(",","")

    # 各データをデータフレームに格納
    se = pd.Series([v_stname,v_ppl], columns)
    df_ppl = df_ppl.append(se, columns)

# ラッピング単位のクラスを取得2
table  = soup.find_all("table", {"class": "passengerTable fr"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_stname = dataRow[1]+"駅"
    v_ppl = dataRow[4].replace(",","")

    # 各データをデータフレームに格納
    se = pd.Series([v_stname,v_ppl], columns)
    df_ppl = df_ppl.append(se, columns)

#TOP100以下
# 親ページURL
url_pr = 'https://www.jreast.co.jp/passenger/2018_0{0}.html'

for i in range(1,6):
  html_doc = urlopen(url_pr.format(i))

  # BeautifulSoupでスクレイピング
  soup = BeautifulSoup(html_doc, 'html.parser')
  # ラッピング単位のクラスを取得1
  table  = soup.find_all("table", {"class": "passengerTable fl"})[0]
  rows = table.findAll("tr")

  for row in rows:
    dataRow = []
    for cell in row.findAll("td"):
      dataRow.append(cell.text)

    #listに何か入っている場合
    if dataRow:

      v_stname = dataRow[0]+"駅"
      v_ppl = dataRow[3].replace(",","")

      # 各データをデータフレームに格納
      se = pd.Series([v_stname,v_ppl], columns)
      df_ppl = df_ppl.append(se, columns)

  # ラッピング単位のクラスを取得2
  table  = soup.find_all("table", {"class": "passengerTable fr"})[0]
  rows = table.findAll("tr")

  for row in rows:
    dataRow = []
    for cell in row.findAll("td"):
      dataRow.append(cell.text)

    #listに何か入っている場合
    if dataRow:

      v_stname = dataRow[0]+"駅"
      v_ppl = dataRow[3].replace(",","")

      # 各データをデータフレームに格納
      se = pd.Series([v_stname,v_ppl], columns)
      df_ppl = df_ppl.append(se, columns)

# 収集したデータをCSV形式で保存
filename = "Train_People.csv"
df_ppl.to_csv(filename, encoding = 'utf-8')
df_ppl.head()

,st,ninzu
0,新宿駅,789366
1,池袋駅,566994
2,東京駅,467165
3,横浜駅,423651
4,品川駅,383442


In [0]:
#乗車人数を結合
df_2 = pd.merge(add_df, df_ppl, on='st' , how='left')#left join
df_2.head()

,date,time,line,from_st,to_st,st,comp_time,etc_time,home_time,morning_time,night_time,cold,hot,warm,yobi_0,yobi_1,yobi_2,yobi_3,yobi_4,yobi_5,yobi_6,ninzu
0,2019/08/10,20:08,八高線,拝島駅,東福生駅,拝島駅,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,30430
1,2019/08/09,22:39,東急東横線,学芸大学駅,学芸大学駅,学芸大学駅,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,NaN
2,2019/08/07,18:00,宇都宮線,東大宮駅,蓮田駅,東大宮駅,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,33351
3,2019/08/06,11:54,総武本線,八街駅,日向駅,八街駅,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,5822
4,2019/08/05,12:18,東急大井町線,自由が丘駅,九品仏駅,自由が丘駅,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,NaN


In [0]:
df_3 = df_2[df_2["ninzu"].isnull()]
df_3["line"].value_counts().head(100)

東武東上線                282
小田急小田原線              199
西武新宿線                185
京王線                  161
京急本線                 153
西武池袋線                147
東武伊勢崎線               136
東急田園都市線              127
近鉄大阪線                115
京成本線                 108
相鉄本線                  79
東武野田線                 71
東急東横線                 65
新京成電鉄新京成線             61
小田急江ノ島線               57
都電荒川線                 51
東京メトロ東西線              44
東北本線                  43
関東鉄道常総線               36
京王井の頭線                36
都営地下鉄新宿線              34
東急大井町線                31
秩父鉄道秩父本線              28
中央本線_(JR東日本)          26
東京メトロ千代田線             23
東急池上線                 22
京王相模原線                21
京成押上線                 19
外房線                   17
西武拝島線                 17
                    ... 
総武本線                  13
東京メトロ半蔵門線             12
都営地下鉄浅草線              12
東葉高速鉄道東葉高速線           11
中央快速線                 11
東京メトロ銀座線               9
相模線                    8
都営地下鉄大江戸線              8
小田急多摩線                 7


In [0]:
profile2 = pdp.ProfileReport(add_df)
profile2.to_file(outputfile="myOutputFile_line2.html")

#html化して表示
HTML(filename='myOutputFile_line2.html')

#XMLファイル読み込み

In [42]:
import xml.etree.ElementTree as ET
import pandas as pd

def xml2df(xml_data):
    root = ET.XML(xml_data) # element tree
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for subchild in child:
            record[subchild.tag] = subchild.text
        all_records.append(record)
    df = pd.DataFrame(all_records)
    return df

# load XML to dataframe (gotta be small)
xml_data = open('S12-18.xml').read()
df = xml2df(xml_data)
df.dtypes

df = df[["{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}stationName","{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}passengers2017"]]
df

,{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}stationName,{http://nlftp.mlit.go.jp/ksj/schemas/ksj-app}passengers2017
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN
